In [47]:
import csv
#from textblob.classifiers import NaiveBayesClassifier
#from nltk import NaiveBayesClassifier, classify
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
import numpy as np
import pandas

import numpy as np
import pandas as pd
import cPickle
from collections import defaultdict
import re

In [26]:
import csv
import re

location_train = "../train.tsv"
location_test = "../test.tsv"

location_train_vw = "rotten.train.vw" #will be created
location_test_vw = "rotten.test.vw" #will be created

#cleans a string "I'm a string!?" returns as "i m a string"
def clean(s):
    return " ".join(re.findall(r'\w+', s,flags = re.UNICODE | re.LOCALE)).lower()

#creates Vowpal Wabbit-formatted file from tsv file
def to_vw(location_input_file, location_output_file, test = False):
    print "\nReading:",location_input_file,"\nWriting:",location_output_file
    with open(location_input_file) as infile, open(location_output_file, "wb") as outfile:
        #create a reader to read train file
        reader = csv.DictReader(infile, delimiter="\t")
        #for every line
        for row in reader:
            #if test set label doesnt matter/or isnt available
            if test:
                label = "1"
            else:
                label = str(int(row['Sentiment'])+1)
                phrase = clean(row['Phrase'])
                outfile.write(   label +
                              " '"+row['PhraseId'] + 
                              " |f " + phrase + 
                              " |a " + 
                              "word_count:"+str(phrase.count(" ")+1)
                              + "\n" )
                
to_vw(location_train, location_train_vw)
to_vw(location_test, location_test_vw, test=True)


Reading: ../train.tsv 
Writing: rotten.train.vw

Reading: ../test.tsv 
Writing: rotten.test.vw


In [30]:
vw  rotten.train.vw -c -k --passes 300 --ngram 7 -b 24 --ect 5 -f rotten.model.vw

SyntaxError: invalid syntax (<ipython-input-30-e5d4008a6200>, line 1)

In [36]:
def scikit_learn(train_set, train_labels):
    text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', OneVsOneClassifier(LinearSVC())),
                     ])
    X_train = np.asarray(train_set)
    text_clf = text_clf.fit(X_train, np.asarray(train_labels))
    return text_clf

In [42]:
data_train = pandas.read_csv('../train.tsv', sep='\t')
data_train.shape

(156060, 4)

In [43]:
data_test = pandas.read_csv('../test.tsv', sep='\t')
data_test.shape

(66292, 3)

In [44]:
    # Train the classifier
    train_set = []
    test_set = []

    ##############################TRAIN DATA#####################
    with open('../train.tsv', 'r') as f:
        sentimentreader = csv.reader(f, delimiter='\t')
        header = sentimentreader.next()
        cnt = 0
        for row in sentimentreader:
            sentence = row[2]
            sentiment = row[3]
            if (cnt < 30):
                test_set.append((sentence, sentiment))
            elif (cnt > 30):
                #cl.train(sentence, sentiment)
                train_set.append((sentence, sentiment))
            cnt += 1
    
    sentence_set = [sentence for sentence, label in train_set]
    label_set = [int(label) for sentence, label in train_set]

    test_sentence = [sentence for sentence, label in test_set]
    label_test = [int(label) for sentence, label in test_set]

    text_clf = scikit_learn(sentence_set, label_set)
    
    #Predict the test data
    #predicted = text_clf.predict(test_sentence)
    #print np.mean(predicted == np.asarray(label_test))
    #for doc, category in zip(test_sentence, predicted):
    #    print('%r => %s' % (doc, category))
    #cl = NaiveBayesClassifier.train(train_set)

    #############################TEST DATA##################
    
    # Read test data and predict phrase based on train set
    with open('../test.tsv', 'r') as f:
        testreader = csv.reader(f, delimiter='\t')
        submission = open('scikit_submission.csv', 'w')
        csvwriter = csv.writer(submission, delimiter=',')
        csvwriter.writerow(['PhraseId', 'Sentiment'])
        header = testreader.next()
        phraseid_list = []
        phrase_list = []
        for row in testreader:
            phraseid = row[0]
            phrase = row[2]
            phraseid_list.append(phraseid)
            phrase_list.append(phrase)
            #rating = cl.classify(phrase, default='0')
            #write_row = [str(phraseid), str(rating)]
            #csvwriter.writerow(write_row)
        
        predicted = text_clf.predict(np.asarray(phrase_list))
        for i in range(len(predicted)):
            sentiment_label = predicted[i]
            phraseid = phraseid_list[i]
            csvwriter.writerow([str(phraseid), str(sentiment_label)])

In [52]:
def build_data_train_test(data_train, data_test, train_ratio = 0.8, clean_string=True):
    """
    Loads data and split into train and test sets.
    """
    revs = []
    vocab = defaultdict(float)
    # Pre-process train data set
    for i in xrange(data_train.shape[0]):
        line = data_train['Phrase'][i]
        y = data_train['Sentiment'][i]
        rev = []
        rev.append(line.strip())
        if clean_string:
            orig_rev = clean_str(' '.join(rev))
        else:
            orig_rev = ' '.join(rev).lower()
        words = set(orig_rev.split())
        for word in words:
            vocab[word] += 1
        datum  = {'y': y, 
                  'text': orig_rev,
                  'num_words': len(orig_rev.split()),
                  'split': int(np.random.rand() < train_ratio)}
        revs.append(datum)
        
    # Pre-process test data set
    for i in xrange(data_test.shape[0]):
        line = data_test['Phrase'][i]
        rev = []
        rev.append(line.strip())
        if clean_string:
            orig_rev = clean_str(' '.join(rev))
        else:
            orig_rev = ' '.join(rev).lower()
        words = set(orig_rev.split())
        for word in words:
            vocab[word] += 1
        datum  = {'y': -1, 
                  'text': orig_rev,
                  'num_words': len(orig_rev.split()),
                  'split': -1}
        revs.append(datum)
        
    return revs, vocab

    
def get_W(word_vecs, k=300):
    """
    Get word matrix. W[i] is the vector for word indexed by i
    """
    vocab_size = len(word_vecs)
    word_idx_map = dict()
    W = np.zeros(shape=(vocab_size+1, k), dtype=np.float32)
    W[0] = np.zeros(k, dtype=np.float32)
    i = 1
    for word in word_vecs:
        W[i] = word_vecs[word]
        word_idx_map[word] = i
        i += 1
    return W, word_idx_map

def load_bin_vec(fname, vocab):
    """
    Loads 300x1 word vecs from Google (Mikolov) word2vec
    """
    word_vecs = {}
    with open(fname, 'rb') as f:
        header = f.readline()
        vocab_size, layer1_size = map(int, header.split())
        binary_len = np.dtype('float32').itemsize * layer1_size
        for line in xrange(vocab_size):
            word = []
            while True:
                ch = f.read(1)
                if ch == ' ':
                    word = ''.join(word)
                    break
                if ch != '\n':
                    word.append(ch)   
            if word in vocab:
                word_vecs[word] = np.fromstring(f.read(binary_len), dtype='float32')  
            else:
                f.read(binary_len)
    return word_vecs

def add_unknown_words(word_vecs, vocab, min_df=1, k=300):
    """
    For words that occur in at least min_df documents, create a separate word vector.    
    0.25 is chosen so the unknown vectors have (approximately) same variance as pre-trained ones
    """
    for word in vocab:
        if word not in word_vecs and vocab[word] >= min_df:
            word_vecs[word] = np.random.uniform(-0.25,0.25,k)  

def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)     
    string = re.sub(r"\'s", " \'s", string) 
    string = re.sub(r"\'ve", " \'ve", string) 
    string = re.sub(r"n\'t", " n\'t", string) 
    string = re.sub(r"\'re", " \'re", string) 
    string = re.sub(r"\'d", " \'d", string) 
    string = re.sub(r"\'ll", " \'ll", string) 
    string = re.sub(r",", " , ", string) 
    string = re.sub(r"!", " ! ", string) 
    string = re.sub(r"\(", " \( ", string) 
    string = re.sub(r"\)", " \) ", string) 
    string = re.sub(r"\?", " \? ", string) 
    string = re.sub(r"\s{2,}", " ", string)    
    return string.strip().lower()

In [53]:
w2v_file = 'GoogleNews-vectors-negative300.bin'

revs, vocab = build_data_train_test(data_train, data_test, train_ratio=0.8, clean_string=True)
max_l = np.max(pd.DataFrame(revs)['num_words'])
print 'data loaded!'
print 'number of sentences: ' + str(len(revs))
print 'vocab size: ' + str(len(vocab))
print 'max sentence length: ' + str(max_l)
print 'loading word2vec vectors...',
w2v = load_bin_vec(w2v_file, vocab)
print 'word2vec loaded!'
print 'num words already in word2vec: ' + str(len(w2v))
add_unknown_words(w2v, vocab)
W, word_idx_map = get_W(w2v)
cPickle.dump([revs, W, word_idx_map, vocab], open('imdb-train-val-test.pickle', 'wb'))
print 'dataset created!'

data loaded!
number of sentences: 222352
vocab size: 17785
max sentence length: 53
loading word2vec vectors...

IOError: [Errno 2] No such file or directory: 'GoogleNews-vectors-negative300.bin'